# Downloading COVID Data From FloridaDisaster.ORG

### To Do

- [ ] **IMPORTANT**: Write the code that allows you to merge RWJ & Census data in
- [ ] **IMPORTANT**: Write code so that the data that gets around the DOH scraping block **SEE**: https://stackoverflow.com/questions/56406466/change-proxy-in-chromedriver-for-scraping-purposes/56515662#56515662, **FREE PROXY** https://sslproxies.org/
- [ ] Figure out why 3-19-20(PM) PDF returns null in the html file
- [ ] Revise the code that imports the html data so that it reads the column headers withou t (Consider using a list with all of the possible values of col head then if found append the list value to header.)
- [ ] *Optional*: Turn all of the processes into functions

### Completed
- [X] **IMPORTANT**: Fix this error '5-31-2020('
- [X] Fix the problem where before doh reported 0 as 0 and not missing, I can't use the data
- [X] Clean the changing column names at the text export stage
- [X] Figure out my headers are messed up
- [X] Why are there so many obs for 3/18
- [X] Extract the page range 12-25
- [X] Find the pages that contain the words "Coronavirus: All persons tested"
- [X] Extract just those two pages pdf 
- [X] Export individual files to txt for storage
- [X] Fix/find out why reports 3/19 - 3/24 are only 1kb and hold no data (There were 3 different page titles not 2)
- [X] **IMPORTANT**: THE LARGE NEGATIVE NUMBERS FOR DADE DO NOT SEEM TO BE CORRECT. FIND OUT WHAT IS GOING ON THERE
- [X] **IMPORTANT**: Something happens to the data at 7/6 after this point everything is wrong, why???



In [ ]:
from selenium import webdriver
import os
import wget
import PyPDF2 as pdf
from PyPDF2 import PdfFileReader, PdfFileWriter
import time
import textract
from pathlib import Path
import re
import pandas as pd
import datetime

In [ ]:
chrome_path = r"chromedriver.exe"

## Initializing driver element and finding 

In [ ]:
dir(chrome_options)

In [ ]:
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
# chrome_options.add_argument("--headless")
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--disable-plugins-discovery")
chrome_options.add_argument("--start-maximized")
driver = webdriver.Chrome(chrome_path,options=chrome_options)

In [ ]:
driver = webdriver.Chrome(chrome_path,options=chrome_options)

In [ ]:
print(driver.execute_script('return navigator.userAgent'))
driver.delete_all_cookies()

In [ ]:
driver.get('https://floridadisaster.org/covid19/covid-19-data-reports/')
title = driver.title
time.sleep(3)
result = driver.find_elements_by_xpath("//a[@href]")
report_months= ['June Reports', 'May Reports', 'April Reports', 'March Reports']
expanded = 0
if expanded == 0:
    for item in result:
        for months in report_months:
            if item.text.find(months) >= 0:
                item.click()
                expanded = 1
time.sleep(3)
result_full = driver.find_elements_by_xpath("//a[@href]")

In [ ]:
for items in result_full:
    print(items.get_attribute('href'))
    break

Initializing the output paths

In [10]:
output_path = Path('../data/PDF')
pdf_output_path= output_path.resolve()
pdf_extract_path = Path('../Data/Unclean PDF').resolve()


In [ ]:
pdf_output_path

## Downloads the PDF Files from FloridaDisaster.ORG

In [ ]:
a = r'https://www.floridadisaster.org/globalassets/covid19/state_reports_20200829.pdf'
b = 'https://files.downloadnow.com/s/software/16/93/95/12/winrar-x64-591.exe?token=1598798865_3b53597d72f1ae86776e3e91a3b8bb37&fileName=winrar-x64-591.exe'
wget.download(a,out='test.pdf')

In [ ]:
print('Starting Data Download')
link_counter = 0
download_counter = 0 
link_n = len(result_full) -152
download_list = []
link_list = []
file_list = []
for links in result_full:
    if links.text.find('Data Report') > 0:
        link_url = links.get_attribute('href')
        filename = wget.filename_from_url(link_url)
        link_list.append(link_url)
        file_list.append(filename)
        test = dict(zip(file_list, link_list))

        
#         if not os.path.exists(f'{pdf_output_path}/{filename}'):
#             wget.download(link_url, out = f'{pdf_output_path}')
#             download_counter += 1
#             download_list.append
#             print("Downloading", links.text)
#         link_counter +=1
#         print (f'{round((link_counter)*100/link_n,2)}% Complete')
# print('Download of New Files Complete')
# print(f'{download_counter} Files Created')

In [ ]:
for items in file_list:
    wget.download(test[items])
    break

In [ ]:
test = dict(zip(file_list, link_list))

In [ ]:
test

In [ ]:
print('Starting Data Download')
link_counter = 0
download_counter = 0 
link_n = len(result_full) -152
download_list = []
for links in result_full:
    if links.text.find('Data Report') > 0:
        link_url = links.get_attribute('href')
        filename = wget.filename_from_url(link_url)
        if not os.path.exists(f'{pdf_output_path}/{filename}'):
            wget.download(link_url, out = f'{pdf_output_path}')
            sleep(2)
            download_counter += 1
            download_list.append
            print("Downloading", links.text)
        link_counter +=1
        print (f'{round((link_counter)*100/link_n,2)}% Complete')
print('Download of New Files Complete')
print(f'{download_counter} Files Created')

In [ ]:
driver.close()

### Extracting the min and max page range for the COVID-19 Daily County Data

In [14]:
from pathlib import Path
import datetime
output_path = Path('../data/PDF')
pdf_output_path= output_path.resolve()
pdf_extract_path = Path('../Data/Unclean PDF').resolve()

In [21]:
#turn into a function
def PdfPageExtract(input_path, output_path, page_num):
    import time
    import os
    import PyPDF2 as pdf
    from PyPDF2 import PdfFileReader, PdfFileWriter
    import textract
    import re
    pdf_extract_list = []
    date_fmt = r'(^.+\D)_(\d+)_(\d+)(\.pdf\Z)'
    print('Program Started')
    page_counter = 0
    today = datetime.date.today()
    file_num = len(os.listdir(input_path))
    file_exist_n =0
    progress = 0
    for files in os.listdir(input_path):
        if re.search(date_fmt, files ):
#             print(files) 
            mod_files = re.sub(date_fmt, r'\1 \2-\3-2020.pdf', files )
#             print(mod_files)    
            output = f'{output_path}/(7-11-20) Unclean Extract from {mod_files}'
        else:
            output = f'{output_path}/(7-11-20) Unclean Extract from {files}'
        progress += 1
        if os.path.exists(output):
            file_exist_n +=1
            print(f'{round((progress/file_num)* 100,2)}%', 'Complete', 'File exists')
            continue
        else:
            print('Creating the file', output)
            pdf_writer = PdfFileWriter()
            pdf_file = open(f'{input_path}/{files}','rb')
            pdfreader = PdfFileReader(pdf_file)
            num_pages = pdfreader.numPages
            if num_pages < page_num:
                up_limit = num_pages
            else:
                up_limit = page_num
            for pages in range(3, up_limit):
                page =  pdfreader.getPage(pages)  
                if page.extractText().find('COVID-19: All persons with tests reported') > 0 or page.extractText().find('Coronavirus: PUI testing by county') > 0 or page.extractText().find('Coronavirus: All persons tested') > 0 or page.extractText().find('Coronavirus: PUI testing by county') > 0 or page.extractText().find('Coronavirus: All') > 0 or page.extractText().find('COVID-19: All') > 0:
                    pdf_writer.addPage(page) 
            
            with open(output, 'wb') as output_pdf:
                pdf_writer.write(output_pdf)
            pdf_extract_list.append(files)
            print(f'{round((progress/file_num)* 100,2)}% ', 'Complete' , 'File created')
    
    if file_exist_n == file_num:
        print('All Files Previously Created')
    else:
        print(f'{file_num -file_exist_n} New Files Created')
    return pdf_extract_list

page_limit = 40

PdfPageExtract(pdf_output_path, pdf_extract_path, page_limit)


Program Started
0.5% Complete File exists
1.0% Complete File exists
1.49% Complete File exists
1.99% Complete File exists
2.49% Complete File exists
2.99% Complete File exists
3.48% Complete File exists
3.98% Complete File exists
4.48% Complete File exists
4.98% Complete File exists
5.47% Complete File exists
5.97% Complete File exists
6.47% Complete File exists
6.97% Complete File exists
7.46% Complete File exists
7.96% Complete File exists
8.46% Complete File exists
8.96% Complete File exists
9.45% Complete File exists
9.95% Complete File exists
10.45% Complete File exists
10.95% Complete File exists
11.44% Complete File exists
11.94% Complete File exists
12.44% Complete File exists
12.94% Complete File exists
13.43% Complete File exists
13.93% Complete File exists
14.43% Complete File exists
14.93% Complete File exists
15.42% Complete File exists
15.92% Complete File exists
16.42% Complete File exists
16.92% Complete File exists
17.41% Complete File exists
17.91% Complete File exist

['state_reports_20200802.pdf']

In [7]:
today = datetime.date.today()
file_list = PdfPageExtract(pdf_output_path, pdf_extract_path, page_limit)
print(f'List of PDF file extracts created {today}')
print('_'*100)
for files in file_list:
    print(files)

Program Started
0.5% Complete File exists
1.0% Complete File exists
1.49% Complete File exists
1.99% Complete File exists
2.49% Complete File exists
2.99% Complete File exists
3.48% Complete File exists
3.98% Complete File exists
4.48% Complete File exists
4.98% Complete File exists
5.47% Complete File exists
5.97% Complete File exists
6.47% Complete File exists
6.97% Complete File exists
7.46% Complete File exists
7.96% Complete File exists
8.46% Complete File exists
8.96% Complete File exists
9.45% Complete File exists
9.95% Complete File exists
10.45% Complete File exists
10.95% Complete File exists
11.44% Complete File exists
11.94% Complete File exists
12.44% Complete File exists
12.94% Complete File exists
13.43% Complete File exists
13.93% Complete File exists
14.43% Complete File exists
14.93% Complete File exists
15.42% Complete File exists
15.92% Complete File exists
16.42% Complete File exists
16.92% Complete File exists
17.41% Complete File exists
17.91% Complete File exist

# Converting the PDFs to html files

In [1]:
from pathlib import Path

input_path = Path('../Data/Unclean PDF')
output_path = Path('../Data/html')
ip = input_path.resolve()
op = output_path.resolve()

In [2]:
import os
import pdftotree
import re
import sys
print('Program started')
html_created_list= []
file_n = len(os.listdir(ip))
file_ct = 0
file_exist_n =0
error_list = []
date1 = r'^.*(\d{2})-*(\d{2})-*(\d{4}).*\Z'
date2 = r'^.*(\d+)_(\d+)_(\d+).*\Z'
date3 = r'^.*(\d{4})-*(\d{2})-*(\d{2}).*\Z'
date4 = r'^.*\D(\d)\.(\d+)\.(\d+).*\Z'
date5 = r'^.*(\d{2})_(\d{2})\Z'
date6 = r'^.*-(\d{1})(\d{2})(\d{2})_.*\Z'
# date_format = r'(^.+)\D(\d-)(.*\Z)'
last_file_name = ''
for files in os.listdir(ip):
    file_ct += 1
    if re.search(date3,files):
#     test_name = re.sub(date3, r'html version of FL COVID Rep \2-\3-\1.html',files)
#     if 
        match = 1
        new_file_name = re.sub(date3, r'html version of FL COVID Rep \2-\3-\1.html',files)    
    elif re.search(date1,files):
        match = 2
        new_file_name = re.sub(date1, r'html version of FL COVID Rep \1-\2-\3.html',files)
    elif re.search(date2,files):
        match = 3        
        new_file_name = re.sub(date2, r'html version of FL COVID Rep 0\1-\2-\3.html',files)
    elif re.search(date4,files):        
        match = 4
        new_file_name = re.sub(date4, r'html version of FL COVID Rep 0\1-\2-2020.html',files)
    elif re.search(date5,files):
        match = 5
        new_file_name = re.sub(date5, r'html version of FL COVID Rep \1-\2-2020.html',files)
    elif re.search(date6,files):
        match = 6
        new_file_name = re.sub(date6, r'html version of FL COVID Rep 0\1-\2-2020.html',files)
        match = 7
    else:
        print("!" *36,'ERROR: Date Not found in File name',"!" *36)
        print('_'*100)
        print('Infile path:', ip)
        print('Outfile path:',op)
        print('Current File Path:',files)
        print('Current File name:',new_file_name)
        print('Prevous File name:',last_file_name)
        sys.exit()
        
    
    new_file_name = re.sub('20-20-0606', '06-06-2020', new_file_name)  
#     new_file_name = re.sub(date_format, r'\1-0\2\3', new_file_name)


    if last_file_name == new_file_name and new_file_name.find('PM') < 0:
        new_file_name = new_file_name.replace('.html', '(PM).html')
    in_file_path = f'{ip}/{files}'
    
    output_file_path = f'{op}\\{new_file_name}.html'
    if os.path.exists(output_file_path):
        file_exist_n +=1
        last_file_name = new_file_name
    else:
        html_created_list.append(output_file_path)
#         print('The infile path is:', in_file_path)
#         print('Creating the html file:', new_file_name, match)
        html = pdftotree.parse(in_file_path)
        with open(output_file_path,'w') as f:
            if html is None:
                print(new_file_name)
            else:
                f.write(html)
            
        last_file_name = new_file_name

    print(f'{round((file_ct/file_n)*100,2)}% Complete') 
if file_exist_n == file_n:
    print('All Files Previously Created')
    print('Program Completed')
else:
    print(f'{file_n - file_exist_n} New Files Created')
    print('Files Created List')
    print('-'*100)
    for files in html_created_list:
        print(files)
    print('Program Completed')
    

Program started
0.49% Complete
0.98% Complete
1.47% Complete
1.96% Complete
2.45% Complete
2.94% Complete
3.43% Complete
3.92% Complete
4.41% Complete
4.9% Complete
5.39% Complete
5.88% Complete
6.37% Complete
6.86% Complete
7.35% Complete
7.84% Complete
8.33% Complete
8.82% Complete
9.31% Complete
9.8% Complete
10.29% Complete
10.78% Complete
11.27% Complete
11.76% Complete
12.25% Complete
12.75% Complete
13.24% Complete
13.73% Complete
14.22% Complete
14.71% Complete
15.2% Complete
15.69% Complete
16.18% Complete
16.67% Complete
17.16% Complete
17.65% Complete
18.14% Complete
18.63% Complete
19.12% Complete
19.61% Complete
20.1% Complete
20.59% Complete
21.08% Complete
21.57% Complete
22.06% Complete
22.55% Complete
23.04% Complete
23.53% Complete
24.02% Complete
24.51% Complete
25.0% Complete
25.49% Complete
25.98% Complete
26.47% Complete
26.96% Complete
27.45% Complete
27.94% Complete
28.43% Complete
28.92% Complete
29.41% Complete
29.9% Complete
30.39% Complete
30.88% Complete
31

In [12]:
type(html)

NoneType

## List of PDF File Extracts

In [ ]:
for files in os.listdir(ip):
    print(files)

## List of html Files


In [ ]:
for files in os.listdir(op):
    print(files)

### Need to fix at some point
>This code currentlyt does not work converting it to a function will allow for easier reuse

In [ ]:
# def HtmlMaker(input_path, output_path, output_name):
#     import os
#     import pdftotree
#     import re
#     print('Program started')
#     file_n = len(os.listdir(input_path))
#     file_ct = 0
#     error_list = []
#     search_list = [r'^.*(\d+)-*(\d{2})-*(\d{4}).*\Z', r'^.*(\d+)_(\d+)_(\d+).*\Z', r'^.*(\d{4})-*(\d{2})-*(\d{2}).*\Z', r'^.*(\d)\.(\d{2})\.(\d+).*\Z', r'^.*(\d{2})_(\d{2})\Z']
#     result_list = [fr'html version of {output_name} \2-\3-\1.html', fr'html version of {output_name} \2-\3-\1.html', fr'html version of {output_name} \1-\2-\3.html', fr'html version of {output_name} \1-\2-\3.html', fr'html version of {output_name} \1-\2-2020.html']
#     last_file_name = ''

#     file_exist_n = 0
#     for files in os.listdir(input_path):
#         looking = 1
#         file_ct += 1
#         for searches in search_list:
#             counter = 0
#             while looking == 1:
#                 if re.search(searches,files):
#                     new_file_name = re.sub(searches, result_list[counter],files).replace('-20.html','-2020.html')
#                     looking = 0
#                     print(files, 'found it at', counter)
#                 counter += 1
#                 if counter  > 20:
#                     break
#         if last_file_name == new_file_name:
#             new_file_name = new_file_name.replace('.html', '(PM).html')
#         in_file_path = f'{input_path}/{files}'
#         try:
#             html = pdftotree.parse(in_file_path)
#             output_file_path = f'{output_path}\\{new_file_name}.html'
#             print(output_file_path)
#             if os.path.exists(output_file_path):
#                 file_exist_n +=1
#                 continue
#             else:
#                 print('I\'LL SEE YOU IN HELL!!!!')
#                 with open(output_file_path,'w') as f:
#                     f.write(html)
#                 last_file_name = new_file_name
#         except: 
#             error_list.append(files)
#         print(f'{round((file_ct/file_n)*100)}% Complete')
#     if file_exist_n == file_num:
#         print('All Files Previously Created')
#     else:
#         print(f'{file_n -file_exist_n} New Files Created')
            
            
# file_name = 'FL COVID Rep'
# HtmlMaker(ip, op, file_name)

## Importing the html files

In [3]:
import pandas as pd
import time
from bs4 import BeautifulSoup as soup
import os
from pathlib import Path
import datetime

In [4]:
today = datetime.date.today()
html_path = Path('../Data/html').resolve()
txt_path = Path('../Data/TXT2').resolve()

## Converting html to text and storing data



**NEED** To fix the promblem where 3-21 is being written with 3-19 & 3-20 included

In [5]:
import sys

header1 = ['CUNTY', 'AWAITING_TESTING', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME']
header1a = ['COUNTY', 'AWAITING_TESTING', 'NEGATIVE', 'POSITIVE',  'TOTAL_TESTED','PERCENT_POSITIVE', 'FILE_NAME']
header2 = ['COUNTY', 'AWAITING_TESTING', 'NEGATIVE', 'POSITIVE', 'INCONCLUSIVE',  'TOTAL_TESTED', 'PERCENT_POSITIVE','FILE_NAME']
header3 = ['COUNTY', 'AWAITING_TESTING', 'INCONCLUSIVE', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED','FILE_NAME']
files_w2 =[]
final_df = pd.DataFrame(columns=header1)
file_counter = 0
file_len_error = []
for files in os.listdir(html_path):
    file_path = f'{html_path}/{files}'
    file =open(file_path,'r')
    df = pd.DataFrame()
    file_n = len(os.listdir(html_path))
    page_soup = soup(file, 'lxml')
    table_soup = page_soup.findAll("table")
    row_counter = 0
    head_use = 999
    for tables in range(0, len(table_soup)):
        table_html = table_soup[tables]
#         print(files)
        for row in table_html.findAll('tr'):
            data_written = 0
            row_length = len(row)
            counter = 1
            data_list = []
            for col in row.findAll('td'):
                
                if row_counter == 0:
                    data_list.append(col.text.replace(' ', '_').upper())
                
                else:
                    data_list.append(col.text.replace(',','').replace('%',''))
                
                if len(data_list) == row_length:     
                    data_list.append(files.replace('.html','').strip('\n'))     
                    if len(data_list) == 7:
                        if row_counter == 0 and file_counter < 4:
                            if files.find("03-20-2020") >= 0:
                                df = pd.DataFrame(columns=header1a)
                            else:
                                df = pd.DataFrame(columns=header1)
                            head_use = 1
                        
                        elif data_written == 0 and row_counter > 0 and data_list[0] != 'County' and len(data_list[0]) != 0 and data_list[0].find('or by mai') < 0:
                            if files.find("03-20-2020") >= 0:
                                data = pd.DataFrame([data_list], columns = header1a)
                            else:
                                data = pd.DataFrame([data_list], columns = header1)
                            df = pd.concat([df,data])
                            data_written = 1

                    elif len(data_list) > 7:
                        if row_counter == 0 and file_counter < 13 and file_counter > 3:
                            df = pd.DataFrame(columns=header2)
                            files_w2.append(files)
                            head_use = 2             
                        elif row_counter == 0:
                            df = pd.DataFrame(columns=header3)
                            head_use = 3
                            
                
                            
                        if data_written == 0 and row_counter > 0 and data_list[0] != 'County' and len(data_list[0]) != 0 and data_list[0].find('or by mai') < 0:
                            if files.replace('.html','').strip('\n').find('html version of FL COVID Rep 03-24-2020') >= 0 and len(data_list) == 9:
                                
                                data_list.pop(4)
                                
                                if head_use == 2:
                                    data = pd.DataFrame([data_list], columns = header2)
                                    df = pd.concat([df,data])
                                    data_written = 1
                                else:
                                    data = pd.DataFrame([data_list], columns = header3)
                                    df = pd.concat([df,data])
                                    data_written = 1

                            else:
    
                                if len(data_list)== 9 and file_counter > 10 and file_counter <36:
                                    data_list.pop(3) # This is needed because the hmtl adds a column before inconclusive for files 12-35
                                if data_written == 0 and head_use == 2:
                                    data = pd.DataFrame([data_list], columns = header2)
                                    df = pd.concat([df,data])
                                    data_written = 1
                                elif data_written == 0 :
    #                                     print(file_counter, data_list)
                                    data = pd.DataFrame([data_list], columns = header3)
                                    df = pd.concat([df,data])
                                    data_written = 1
                                else:
                                    print('No data written Data lines =', data_list)
                    
        
            row_counter += 1
#     outfile = f'{txt_path}/{files.replace("html.html", "txt").replace("html version", "txt version")}'
    outfile1 = f'{txt_path}/{files.replace("html.html", "csv").replace("html version", "txt version")}'
    file_counter +=1
#     df.to_csv(outfile,sep='\t', index = False)
    
    try:
        if df['COUNTY'].count() > 70 or df['COUNTY'].count() < 57 :
            n = df['COUNTY'].count()
            file_len_error.append(file)
            print('\n'*5)
            print('ERORR '*10)
            print('df OBS Count Flag!!!', files)
            print(f'There are {n} observations in this file')
            print('ERORR '*10)
            print('\n'*5)
        else:
            final_df = final_df.append(df, ignore_index=True)
            df.to_csv(outfile1,sep=',', index = False)
    except KeyError:
        print('\n'*5)
        print('ERORR '*10)
        print('file', files, 'has no data.')
        print('ERORR '*10)
        print('\n'*5)
        file_len_error.append(file)
    print(f'{round((file_counter/file_n)*100,3)}% Complete')


    







ERORR ERORR ERORR ERORR ERORR ERORR ERORR ERORR ERORR ERORR 
file html version of FL COVID Rep 03-16-2020.html.html has no data.
ERORR ERORR ERORR ERORR ERORR ERORR ERORR ERORR ERORR ERORR 






0.498% Complete






ERORR ERORR ERORR ERORR ERORR ERORR ERORR ERORR ERORR ERORR 
file html version of FL COVID Rep 03-18-2020.html.html has no data.
ERORR ERORR ERORR ERORR ERORR ERORR ERORR ERORR ERORR ERORR 






0.995% Complete






ERORR ERORR ERORR ERORR ERORR ERORR ERORR ERORR ERORR ERORR 
file html version of FL COVID Rep 03-19-2020(PM).html.html has no data.
ERORR ERORR ERORR ERORR ERORR ERORR ERORR ERORR ERORR ERORR 






1.493% Complete






ERORR ERORR ERORR ERORR ERORR ERORR ERORR ERORR ERORR ERORR 
file html version of FL COVID Rep 03-19-2020.html.html has no data.
ERORR ERORR ERORR ERORR ERORR ERORR ERORR ERORR ERORR ERORR 






1.99% Complete
2.488% Complete
2.985% Complete






ERORR ERORR ERORR ERORR ERORR ERORR ERORR ERORR ERORR ERORR 
file html version of FL CO

In [ ]:
for files in files_w2:
    print(files)

## QA Check on Data Import Process

In [8]:
print(f'There are {len(file_len_error)} html files with too few or too many observations')
print(f'Date of last run: {today}')
print('File Names:')
print('-'*92)
for files in file_len_error:
    print(files.name)
print('NOTE: Files listed here will not appear in the final dataset')    


There are 7 html files with too few or too many observations
Date of last run: 2020-08-30
File Names:
--------------------------------------------------------------------------------------------
C:\Users\chacr\OneDrive\Documents\Coding\COVID-19 Project\Data\html/html version of FL COVID Rep 03-16-2020.html.html
C:\Users\chacr\OneDrive\Documents\Coding\COVID-19 Project\Data\html/html version of FL COVID Rep 03-18-2020.html.html
C:\Users\chacr\OneDrive\Documents\Coding\COVID-19 Project\Data\html/html version of FL COVID Rep 03-19-2020(PM).html.html
C:\Users\chacr\OneDrive\Documents\Coding\COVID-19 Project\Data\html/html version of FL COVID Rep 03-19-2020.html.html
C:\Users\chacr\OneDrive\Documents\Coding\COVID-19 Project\Data\html/html version of FL COVID Rep 03-21-2020.html.html
C:\Users\chacr\OneDrive\Documents\Coding\COVID-19 Project\Data\html/html version of FL COVID Rep 07-26-2020.html.html
C:\Users\chacr\OneDrive\Documents\Coding\COVID-19 Project\Data\html/html version of FL COVID 

Exporting the COVID Dataset

Setting Paths for output

In [9]:
qa_path = '../data/QA Checks/'
graphics_path = '../Graphics/'
data_path = '../data/'

In [10]:
final_df.loc[final_df['COUNTY'].str.contains('Dade') ,'COUNTY'] = 'Miami-Dade'

In [11]:
final_df.loc[final_df['COUNTY'].str.contains('Dade')]

,CUNTY,AWAITING_TESTING,NEGATIVE,POSITIVE,PERCENT_POSITIVE,TOTAL_TESTED,FILE_NAME,COUNTY,INCONCLUSIVE
1,NaN,113,838,123,13,961,html version of FL COVID Rep 03-20-2020,Miami-Dade,NaN
67,NaN,97,1211,169,12,1381,html version of FL COVID Rep 03-21-2020(PM),Miami-Dade,1
134,NaN,93,1480,228,13,1709,html version of FL COVID Rep 03-22-2020(PM),Miami-Dade,1
202,NaN,82,1246,177,12,1424,html version of FL COVID Rep 03-22-2020,Miami-Dade,1
268,NaN,91,1845,278,13,2124,html version of FL COVID Rep 03-23-2020(PM),Miami-Dade,1
...,...,...,...,...,...,...,...,...,...
13034,NaN,461,631260,152446,19,785674,html version of FL COVID Rep 08-25-2020,Miami-Dade,1968
13103,NaN,472,635483,152939,19,790393,html version of FL COVID Rep 08-26-2020,Miami-Dade,1971
13172,NaN,473,638772,153555,19,794298,html version of FL COVID Rep 08-27-2020,Miami-Dade,1971
13241,NaN,469,642200,154216,19,798391,html version of FL COVID Rep 08-28-2020,Miami-Dade,1975


In [12]:
final_df.to_excel(f'{data_path}Final COVID Only Dataset (Pre-Manipulation) 7-13-20.xlsx', index=False)

Importing the COVID Dataset

In [ ]:
# mod_df = pd.read_excel(f'{data_path}Final COVID Only Dataset (Pre-Manipulation) 7-13-20.xlsx')


In [14]:
import pandas as pd
mod_df = final_df.loc[~final_df['COUNTY'].str.contains('Total')].copy()
mod_df = mod_df.loc[~mod_df['COUNTY'].str.contains('Total')]
mod_df = mod_df.loc[~mod_df['COUNTY'].str.contains('Total')]
mod_df[mod_df['COUNTY']=='Total']

,CUNTY,AWAITING_TESTING,NEGATIVE,POSITIVE,PERCENT_POSITIVE,TOTAL_TESTED,FILE_NAME,COUNTY,INCONCLUSIVE


In [15]:
mod_df['COUNTY'].nunique()

69

Number of times COUNTY contained 'Sumter'

In [ ]:
# pd.value_counts(mod_df['COUNTY'].str.contains('Sumter')).to_frame()

# Data Manipulation

In [16]:
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns

## Data Cleaning

In [17]:
mod_df['NEGATIVE'] = pd.to_numeric(mod_df['NEGATIVE'])
mod_df['POSITIVE'] = pd.to_numeric(mod_df['POSITIVE'])
mod_df['PERCENT_POSITIVE'] = pd.to_numeric(mod_df['PERCENT_POSITIVE'])
mod_df['TOTAL_TESTED'] = pd.to_numeric(mod_df['TOTAL_TESTED'])
mod_df['INCONCLUSIVE'] = pd.to_numeric(mod_df['INCONCLUSIVE'])
mod_df['AWAITING_TESTING'] = pd.to_numeric(mod_df['AWAITING_TESTING'])

In [18]:
mod_df = mod_df.fillna(0)
mod_df['FILE_NAME'] =  mod_df['FILE_NAME'].str.strip('\n')
mod_df['DATE'] = pd.to_datetime(mod_df['FILE_NAME'].str[29:39], errors='ignore')
mod_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13185 entries, 0 to 13377
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   CUNTY             13185 non-null  int64         
 1   AWAITING_TESTING  13185 non-null  float64       
 2   NEGATIVE          13185 non-null  float64       
 3   POSITIVE          13185 non-null  float64       
 4   PERCENT_POSITIVE  13185 non-null  float64       
 5   TOTAL_TESTED      13185 non-null  int64         
 6   FILE_NAME         13185 non-null  object        
 7   COUNTY            13185 non-null  object        
 8   INCONCLUSIVE      13185 non-null  float64       
 9   DATE              13185 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(5), int64(2), object(2)
memory usage: 1.1+ MB


In [19]:
mod_df['DAY_OF_WEEK'] = mod_df['DATE'].dt.weekday

## Evaluation of the number of times a date appears in the dataset

In [20]:
print(pd.value_counts(mod_df['DATE']).to_frame(name='Frequency of Each Date in the Data set').head(10))
print(pd.value_counts(mod_df['DATE']).to_frame(name='Frequency of Each Date in the Data set').tail(10))
print('NOTE: Each Date should only appear max = 136 times min = 57 times')

            Frequency of Each Date in the Data set
2020-04-18                                     136
2020-03-26                                     136
2020-04-07                                     136
2020-04-20                                     136
2020-04-02                                     136
2020-04-15                                     136
2020-03-28                                     136
2020-04-10                                     136
2020-04-23                                     136
2020-04-05                                     136
            Frequency of Each Date in the Data set
2020-08-15                                      68
2020-06-19                                      68
2020-08-02                                      68
2020-06-06                                      68
2020-06-13                                      68
2020-05-24                                      68
2020-08-09                                      68
2020-05-23                     

### Frequency of county n

In [21]:
print(pd.value_counts(mod_df['COUNTY']).to_frame(name='Frequency of County Name Head').head(10))
print(pd.value_counts(mod_df['COUNTY']).to_frame(name='Frequency of County Name Tail').tail(10))

              Frequency of County Name Head
Brevard                                 194
Nassau                                  194
St. Lucie                               194
Charlotte                               194
Bay                                     194
Seminole                                194
Gulf                                    194
Indian River                            194
Jackson                                 194
Citrus                                  194
                   Frequency of County Name Tail
Lafayette                                    194
Gadsden                                      194
Highlands                                    194
Unknown                                      194
Collier                                      194
Calhoun                                      194
Suwannee                                     194
Liberty                                      188
Glades                                       188
State-Call Center         

In [22]:
county_n = mod_df['COUNTY'].value_counts()
print(county_n.value_counts().to_frame(name='Distribution of County Frequency'))
county_n.to_excel(f'{qa_path}Number of Obs Per County 7-11-20.xlsx')

     Distribution of County Frequency
194                                66
188                                 2
5                                   1


comparing the number of different lengths for filename there are

## Data Quality Check

In [23]:
mod_df['len_file_name'] = pd.value_counts(mod_df['FILE_NAME'].apply(len))
pd.value_counts(mod_df['FILE_NAME'].apply(len)).to_frame(name='Freq of File Lengths')

,Freq of File Lengths
39,10743
43,2306
38,136


In [24]:
pd.value_counts(mod_df['FILE_NAME'].str.contains('PM')).to_frame()

,FILE_NAME
False,10879
True,2306


In [25]:
mod_df['FILE_NAME_LEN'] = mod_df.FILE_NAME.apply(len)

In [26]:
mod_df.sort_values(by=['FILE_NAME_LEN'], inplace=True)

In [27]:
mod_df[["FILE_NAME", 'FILE_NAME_LEN']].head(10)

,FILE_NAME,FILE_NAME_LEN
7801,html version of FL COVID Rep 06-1-2020,38
7815,html version of FL COVID Rep 06-1-2020,38
7814,html version of FL COVID Rep 06-1-2020,38
7813,html version of FL COVID Rep 06-1-2020,38
7812,html version of FL COVID Rep 06-1-2020,38
7811,html version of FL COVID Rep 06-1-2020,38
7810,html version of FL COVID Rep 06-1-2020,38
7809,html version of FL COVID Rep 06-1-2020,38
7808,html version of FL COVID Rep 06-1-2020,38
7816,html version of FL COVID Rep 06-1-2020,38


In [28]:
mod_df[["FILE_NAME", 'FILE_NAME_LEN']].tail(10)

,FILE_NAME,FILE_NAME_LEN
1621,html version of FL COVID Rep 04-02-2020(PM),43
1620,html version of FL COVID Rep 04-02-2020(PM),43
1619,html version of FL COVID Rep 04-02-2020(PM),43
1618,html version of FL COVID Rep 04-02-2020(PM),43
1617,html version of FL COVID Rep 04-02-2020(PM),43
1616,html version of FL COVID Rep 04-02-2020(PM),43
1615,html version of FL COVID Rep 04-02-2020(PM),43
1614,html version of FL COVID Rep 04-02-2020(PM),43
3116,html version of FL COVID Rep 04-13-2020(PM),43
1232,html version of FL COVID Rep 03-30-2020(PM),43


In [29]:
mod_df[mod_df['FILE_NAME_LEN']== 39].filter(['FILE_NAME_LEN','FILE_NAME']).head(10)

,FILE_NAME_LEN,FILE_NAME
8970,39,html version of FL COVID Rep 06-25-2020
8969,39,html version of FL COVID Rep 06-25-2020
8968,39,html version of FL COVID Rep 06-25-2020
8963,39,html version of FL COVID Rep 06-25-2020
8966,39,html version of FL COVID Rep 06-25-2020
8965,39,html version of FL COVID Rep 06-25-2020
8964,39,html version of FL COVID Rep 06-25-2020
8967,39,html version of FL COVID Rep 06-25-2020
8971,39,html version of FL COVID Rep 06-25-2020
8981,39,html version of FL COVID Rep 06-25-2020


In [30]:
mod_df[mod_df['FILE_NAME_LEN']== 39].filter(['FILE_NAME_LEN','FILE_NAME']).tail(10)

,FILE_NAME_LEN,FILE_NAME
5168,39,html version of FL COVID Rep 05-01-2020
5166,39,html version of FL COVID Rep 05-01-2020
5060,39,html version of FL COVID Rep 04-29-2020
5157,39,html version of FL COVID Rep 04-30-2020
5061,39,html version of FL COVID Rep 04-29-2020
5062,39,html version of FL COVID Rep 04-29-2020
5063,39,html version of FL COVID Rep 04-29-2020
5064,39,html version of FL COVID Rep 04-29-2020
5065,39,html version of FL COVID Rep 04-29-2020
5066,39,html version of FL COVID Rep 04-29-2020


## Creating the New Case And Negative Variables

In [31]:
mod_df.sort_values(by=["COUNTY", "DATE"], inplace=True)

In [32]:
mod_df['LAST_POS_COUNT'] = mod_df['POSITIVE'].shift(1)
mod_df['LAST_COUNTY'] = mod_df['COUNTY'].shift(1)
mod_df['LAST_NEG_COUNT'] = mod_df['NEGATIVE'].shift(1)
mod_df['NEW_NEGATIVES'] = mod_df['NEGATIVE'] - mod_df['LAST_NEG_COUNT']
mod_df['NEW_CASES'] = mod_df['POSITIVE'] - mod_df['LAST_POS_COUNT']
mod_df.loc[mod_df['COUNTY'] != mod_df["LAST_COUNTY"], 'NEW_CASES' ] = ''
mod_df.loc[mod_df['COUNTY'] != mod_df["LAST_COUNTY"], 'NEW_NEGATIVES' ] = ''

mod_df['NEW_CASES'] = pd.to_numeric(mod_df['NEW_CASES'])
mod_df['NEW_NEGATIVES'] = pd.to_numeric(mod_df['NEW_NEGATIVES'])
# mod_df.drop(labels=['LAST_POS_COUNT', 'LAST_COUNTY','LAST_NEG_COUNT'], inplace=True, axis=1)


In [ ]:
dade = mod_df.loc[mod_df['COUNTY']=='Miami-Dade']
dade.to_csv(f'{qa_path}Dade data.csv', index=False)
sumter = mod_df.loc[mod_df['COUNTY']=='Broward']
sumter.to_csv(f'{qa_path}Broward data.csv', index=False)
print(dade[['NEW_CASES', 'LAST_POS_COUNT', 'POSITIVE', 'DATE']].head(30))
print(dade[['NEW_CASES', 'LAST_POS_COUNT', 'POSITIVE', 'DATE']].tail(30))
# for cases in dade['NEW_CASES']:
#     print(cases, )

In [33]:
mod_df['TOTAL_TESTED'] = mod_df['NEGATIVE']  +  mod_df['POSITIVE']  + mod_df['INCONCLUSIVE'] + mod_df['AWAITING_TESTING']

In [ ]:
# os.getcwd()
# os.listdir('../')

In [34]:
mod_df.columns

Index(['CUNTY', 'AWAITING_TESTING', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE',
       'TOTAL_TESTED', 'FILE_NAME', 'COUNTY', 'INCONCLUSIVE', 'DATE',
       'DAY_OF_WEEK', 'len_file_name', 'FILE_NAME_LEN', 'LAST_POS_COUNT',
       'LAST_COUNTY', 'LAST_NEG_COUNT', 'NEW_NEGATIVES', 'NEW_CASES'],
      dtype='object')

In [35]:
mod_df.drop(labels=['len_file_name', 'FILE_NAME_LEN', 'LAST_POS_COUNT', 'LAST_NEG_COUNT', 'LAST_COUNTY',], axis = 1, inplace = True)

In [36]:
# mod_df.to_csv(f'{data_path}Final COVID Only Dataset {today.month}-{today.day}-{today.year} (CLEAN).txt', sep='\t', index=False)
mod_df.to_excel(f'{data_path}Final COVID Only Dataset {today.month}-{today.day}-{today.year} (CLEAN).xlsx',  index=False)

In [ ]:
plt.style.use('seaborn-muted')
plt.plot(mod_df['TOTAL_TESTED'],mod_df['NEW_CASES'],'bo', label='New Cases')
plt.plot(mod_df['TOTAL_TESTED'],mod_df['NEW_NEGATIVES'], color='#332244',marker='o', linestyle='' , label= 'Negatives', alpha=0.2)
plt.title('Scatter Plot of number of Total Daily Test (X) and Number of New Cases (Y)')
plt.xlabel('Number of Tests')
plt.ylabel('Number of New Cases')
plt.legend()
plt.show()

In [ ]:
mod_df['NEW_CASES'].hist(bins=[0,199,200,399, 400,1000,1001,5000,5001,10000])

In [ ]:
mod_df[['TOTAL_TESTED','COUNTY']].describe()

In [ ]:
mod_df.COUNTY.nunique()

### Verifying that the shift worked correctly

In [ ]:
mod_df[['NEGATIVE']].sum().to_frame()

In [ ]:
mod_df[['NEW_CASES']].sum().to_frame()

In [ ]:
mod_df.columns

In [ ]:
mod_df['NEW_CASES'].describe().to_frame()

In [ ]:
mod_df.filter(items=['NEW_CASES','COUNTY'], axis=1)

## Observations where the number of new cases is less than 0

In [ ]:
# Getting Counties with over 1000 cases reported in one day
high_case = mod_df[mod_df['NEW_CASES']> 1000]
high_case.sort_values(by='NEW_CASES', inplace=True, ascending =False)
print(high_case[['COUNTY', 'NEW_CASES' ,'DATE']].head(100))
top_100 = pd.DataFrame(high_case[['COUNTY', 'NEW_CASES' ,'DATE']].head(100))


In [ ]:
# County
pd.value_counts(high_case['COUNTY'].head(100)).to_frame(name="County")

In [ ]:
neg_case = mod_df[mod_df['NEW_CASES']< 0].copy()
neg_case.sort_values(by='NEW_CASES', inplace=True)
print(neg_case[['COUNTY', 'NEW_CASES' ,'DATE']].head(20))
print(neg_case[['COUNTY', 'NEW_CASES' ,'DATE']].tail(20))
neg_case.to_csv(f'{qa_path}Observation With Negative.csv' ,sep=',')

In [ ]:
pd.value_counts(neg_case.COUNTY)

In [ ]:
neg_c_freq = pd.DataFrame(neg_case.groupby('DATE')['COUNTY'].count())
neg_c_freq['COUNTY']
neg_c_freq['DATE'] = neg_c_freq.index
sns.barplot(x='DATE', y ='COUNTY',data =neg_c_freq )
plt.title('Number of Daily Negative Case By Date')
plt.ylabel('Frequency of Negative Values')

In [ ]:
mod_df

In [ ]:
mod_df[mod_df['NEW_CASES']< 0].describe()

In [ ]:
# mod_df.drop('FILE_NAME1', axis = 1, inplace=True)
mod_df.columns

# Messing around with matplotlib

In [ ]:
# plt.xkcd()
# plt.plot(mod_df['TOTAL_TESTED'],mod_df['NEW_CASES'],'bo', label='New Cases')
# plt.plot(mod_df['TOTAL_TESTED'],mod_df['NEW_NEGATIVES'], color='#332244',marker='o', linestyle='' , label= 'Negatives', alpha=0.2)
# plt.title('Scatter Plot of number of Total Daily Test (X) and Number of New Cases (Y)')
# plt.xlabel('Number of Tests')
# plt.ylabel('Number of New Cases')
# plt.legend()
# plt.savefig('plot.png')
# plt.show()

In [ ]:
today.day